# EJERCICIO PROCESAMIENTO

Para este ejercicio se usaron los ficheros completos (no los small data) de los datos propuestos.

In [1]:
from pyspark import SparkContext
import numpy as np
import funciones as udf

package "funciones" succesfully loaded


Primero se declara la sesión de spark y sendas rutas a los ficheros.

In [2]:
sc = SparkContext(master = 'local[*]')

In [2]:
main_path = '/media/daniel/Seagate Basic/spark_practica1/sensores' # path donde se encuantran los csv
path_accel_wat = f'{main_path}/Watch_accelerometer.csv'
path_accel_mob = f'{main_path}/Phones_accelerometer.csv'
path_gyros_wat = f'{main_path}/Watch_gyroscope.csv'
path_gyros_mob = f'{main_path}/Phones_gyroscope.csv'

Después, se usan las funciones para procesar los datos, siguiendo los siguientes pasos:

1. Separar los valores de cada línea en el string por defecto al realizar la lectura.
2. Pasar a clave valor las observaciones siguiendo la estructura (g1-g2-g3, (valores)).
3. Eliminar las claves que contengan null como segmento de agrupación.

El tercer paso se debe a que se han encontrado valores perdidos ('null') en la variable que define las acciones. Todas las clases que contengan null seran eliminadas porque no tiene sentido una clase con acción indeterminada.

In [17]:
# se crean los RDDs, se preprocesan, se calculan los estadísticos pedidos y se eliminan las claves con missings
rdd_accel_mob = sc.textFile(path_accel_mob)
rdd_accel_mob = udf.preprocesa_rdd(rdd_accel_mob)
accel_mob_agg = udf.summary(rdd_accel_mob)
accel_mob_agg = udf.remove_null_keys(accel_mob_agg)

rdd_accel_wat = sc.textFile(path_accel_wat)
rdd_accel_wat = udf.preprocesa_rdd(rdd_accel_wat)
accel_wat_agg = udf.summary(rdd_accel_wat)
accel_wat_agg = udf.remove_null_keys(accel_wat_agg)

rdd_gyros_mob = sc.textFile(path_gyros_mob)
rdd_gyros_mob = udf.preprocesa_rdd(rdd_gyros_mob)
gyros_mob_agg = udf.summary(rdd_gyros_mob)
gyros_mob_agg = udf.remove_null_keys(gyros_mob_agg)

rdd_gyros_wat = sc.textFile(path_gyros_wat)
rdd_gyros_wat = udf.preprocesa_rdd(rdd_gyros_wat)
gyros_wat_agg = udf.summary(rdd_gyros_wat)
gyros_wat_agg = udf.remove_null_keys(gyros_wat_agg)

join_wat = accel_wat_agg.join(gyros_wat_agg)
join_mob = accel_mob_agg.join(gyros_mob_agg)

union_data = join_wat.union(join_mob)

union_data = union_data.map(lambda x: (x[0], tuple(udf.flatten(x))[1:])) # se suaviza el resultado final

In [11]:
accel_mob_agg.take(1) #(grupo, (media(x,y,z), min(x, y,z), max(x,y,z), std(x, y,z)))

[('d-nexus4-walk',
  (-3.3188981584809225,
   -0.28856426149887004,
   9.450924416874267,
   -16.117615,
   -7.0500793,
   3.3018036,
   3.7467498999999997,
   5.350021400000001,
   17.71753,
   2.271896034546832,
   1.5930168876151198,
   2.478863255473076))]

In [14]:
join_wat.take(1)

[('e-lgwatch-sit',
  ((1.8760753940141008,
    8.093533184880828,
    4.254708372294288,
    -8.994415,
    -12.1167145,
    0.78411865,
    16.294647,
    10.465424,
    8.447128,
    1.669104779711666,
    1.1202746717638736,
    0.45037339414998034),
   (0.003548147151567854,
    0.0004308153084735134,
    0.0011935236748547794,
    -1.7831879,
    -0.49371338,
    -0.20968628,
    2.6858063,
    1.4417114,
    0.90649414,
    0.07831189498622515,
    0.04062564913072338,
    0.030050879271012815)))]

In [6]:
join_wat.count(), join_mob.count()

(101, 162)

In [16]:
union_data.take(1) #los primeros doce valores son los estadisticos para acelerom. y los 12 ultimos para giros.

[('e-lgwatch-sit',
  (1.8760753940141008,
   8.093533184880828,
   4.254708372294288,
   -8.994415,
   -12.1167145,
   0.78411865,
   16.294647,
   10.465424,
   8.447128,
   1.669104779711666,
   1.1202746717638736,
   0.45037339414998034,
   0.003548147151567854,
   0.0004308153084735134,
   0.0011935236748547794,
   -1.7831879,
   -0.49371338,
   -0.20968628,
   2.6858063,
   1.4417114,
   0.90649414,
   0.07831189498622515,
   0.04062564913072338,
   0.030050879271012815))]

In [15]:
union_data.count()

263

# Pruebas variando cores

En esta parte se mete todo el proceso anterior en una función y se ejecuta variando los cores que utiliza Spark. Para cada una de las pruebas se reinicia el kernel de la notebook para no solapar las sesiones de spark.

In [5]:
from time import time

In [7]:
def prueba():
    
    rdd_accel_mob = sc.textFile(path_accel_mob)
    rdd_accel_mob = udf.preprocesa_rdd(rdd_accel_mob)
    accel_mob_agg = udf.summary(rdd_accel_mob)
    accel_mob_agg = udf.remove_null_keys(accel_mob_agg)

    rdd_accel_wat = sc.textFile(path_accel_wat)
    rdd_accel_wat = udf.preprocesa_rdd(rdd_accel_wat)
    accel_wat_agg = udf.summary(rdd_accel_wat)
    accel_wat_agg = udf.remove_null_keys(accel_wat_agg)

    rdd_gyros_mob = sc.textFile(path_gyros_mob)
    rdd_gyros_mob = udf.preprocesa_rdd(rdd_gyros_mob)
    gyros_mob_agg = udf.summary(rdd_gyros_mob)
    gyros_mob_agg = udf.remove_null_keys(gyros_mob_agg)

    rdd_gyros_wat = sc.textFile(path_gyros_wat)
    rdd_gyros_wat = udf.preprocesa_rdd(rdd_gyros_wat)
    gyros_wat_agg = udf.summary(rdd_gyros_wat)
    gyros_wat_agg = udf.remove_null_keys(gyros_wat_agg)

    join_wat = accel_wat_agg.join(gyros_wat_agg)
    join_mob = accel_mob_agg.join(gyros_mob_agg)

    union_data = join_wat.union(join_mob)

    union_data = union_data.map(lambda x: (x[0], tuple(udf.flatten(x))[1:])) # se suaviza el resultado final
    
    return union_data

### 1 core (reiniciar el kernel)

In [4]:
sc = SparkContext(master = 'local[1]')

In [7]:
t0 = time()
prueba().collect()
t1 = time()
print(f'tiempo: {t1-t0}')

tiempo: 1091.4489452838898


### 2 cores (reiniciar el kernel)

In [3]:
sc = SparkContext(master = 'local[2]')

In [8]:
t0 = time()
prueba().collect()
t1 = time()
print(f'tiempo: {t1-t0}')

tiempo: 679.7884542942047


Se observa que al aumentar el número de cores el tiempo de ejecución se reduce entre un 40% y un 50%.